<a href="https://colab.research.google.com/github/kobarmn/LangChain_GoogleColab/blob/main/test_OpenAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -q -U google-genai

In [ ]:
pip install --upgrade openai

In [ ]:
from openai import OpenAI

# 環境変数 (GEMINI_API_KEY) を設定すると、GeminiAPIライブラリを利用する際に、クライアントにより自動取得される。
# client = genai.Client()

client = OpenAI(
    api_key="Your_apikey",
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

response = client.chat.completions.create(
    model="gemini-2.5-flash",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content" : "GeminiAPIについて教えてください。"
        }
    ]
)

print(response.choices[0].message)